In [ ]:
# imports
# basic
import numpy as np
from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_env import TrainState
# for rendering
import PIL
from flatland.utils.rendertools import RenderTool
from IPython.display import clear_output
# for plotting
import matplotlib.pyplot as plt
# for running clingo
import json
import subprocess
import os
import pickle

# Functions

In [ ]:
# render the environment
def render_env(env,wait=True):
    
    env_renderer = RenderTool(env, gl="PILSVG")
    env_renderer.render_env()

    image = env_renderer.get_image()
    pil_image = PIL.Image.fromarray(image)
    pil_image = pil_image.convert("RGB")
    clear_output(wait=True)
    display(pil_image)

## Export

In [ ]:
# get global transitions
def get_transitions(obsx):
    transitions = obsx[0][0]
    x = 0
    atoms = []

    for i in transitions:
        y = 0
        for j in i:
            directions = ["(\"N\",\"N\")", "(\"N\",\"E\")", "(\"N\",\"S\")", "(\"N\",\"W\")", "(\"E\",\"N\")", "(\"E\",\"E\")", "(\"E\",\"S\")", "(\"E\",\"W\")", "(\"S\",\"N\")", "(\"S\",\"E\")", "(\"S\",\"S\")", "(\"S\",\"W\")", "(\"W\",\"N\")", "(\"W\",\"E\")", "(\"W\",\"S\")", "(\"W\",\"W\")"]
            usableDirections = [directions[i] for i, bit in enumerate(j) if bit == 1]

            for k in usableDirections:
                atoms.append(f"transition(({x},{y}),{k})")
            y += 1
        x += 1

    return atoms

In [ ]:
# extract info of an agent
def get_agents(agent):
    atoms = []

    # state(A,(X,Y),T,D)
    # A - agent handle
    # (X,Y) - position of the agent
    # T - time til earliest departure
    # D - direction "N", "E", "S" or "W"
    directions = {0:"\"N\"", 1:"\"E\"", 2:"\"S\"", 3:"\"W\""}
    initialstate = "initialstate(" + str(agent.handle) + ", " + str(agent.initial_position) + ", " + str(agent.earliest_departure) + ", " + str(directions[agent.direction]) + ")"
    atoms.append(initialstate)

    # target(A,(X,Y),T)
    # A - agent handle
    # (X,Y) - position of the target
    # T - time til latest arrival
    target = "target(" + str(agent.handle) + ", "+str(agent.target) + ", " + str(agent.latest_arrival) + ")"
    atoms.append(target)

    # speed(A,S)
    # A - agent handle
    # S - steps taken for one transition
    speed = "speed(" + str(agent.handle) + ", " + str(agent.speed_counter.max_count+1) + ")"
    atoms.append(speed)

    return(atoms)

In [ ]:
# get all atoms combined (agents and transitions)
def get_atoms(env,obs,filename):
    # get atoms
    agent_handles = env.get_agent_handles()
    atoms = get_transitions(obs[agent_handles[0]])
    for i in agent_handles:
        atoms = atoms + get_agents(env.agents[i])

    # export atoms
    atoms.append("")
    output = ". ".join(atoms)
    with open(filename, "w") as file:
        file.write(output)

## Import

In [ ]:
def run_clingo(input, encoding, timeout, ram_limit):
    dirs = os.listdir(encoding)
    list.sort(dirs)
    dirs = [i for i in dirs if "step" in i]

    total_time = 0
    solve_time = 0
    output = input

    for i in dirs:
        # command = "ulimit -v " + str(ram_limit*1024*1024*1024) + "; " + "echo \'" + input + "\' | clingo - " + encoding + i + " --outf=2 | jq"
        # command = "ulimit -v " + str(ram_limit*1024*1024*1024) + "; clingo - " + encoding + i + " --outf=2 | jq"
        command = "clingo - " + encoding + i + " --outf=2 | jq"
        try:
            output = subprocess.check_output(command, shell=True, timeout=timeout, stderr=subprocess.DEVNULL, input=input.encode("utf-8")).decode("utf-8")
        except subprocess.TimeoutExpired:
            return "TIMEOUT", None, None, None
        except subprocess.CalledProcessError:
            return "MEMORY", None, None, None

        try:
            data = json.loads(output)
        except json.JSONDecodeError:
            return "MEMORY", None, None, None

        if data["Result"] == "UNKNOWN":
            return "MEMORY", None, None, None
        if data["Result"] == "UNSATISFIABLE":
            return "UNSATISFIABLE", None, None, None
        
        total_time += data["Time"]["Total"]
        solve_time += data["Time"]["Solve"]
        output = data["Call"][-1]["Witnesses"][0]["Value"]
        input = output.copy()

        input.append("")
        input = ".".join(input)
        
    return "SATISFIABLE", total_time, solve_time, output


def facts_to_flatland(atoms):
    dictionaries = {}

    atoms = [a for a in atoms if "outputaction" in a]

    # Iterate through the values
    for atom in atoms:
        parts = atom.split(",")
        x = int(parts[0].split("(")[1])
        a = int(parts[1])
        t = int(parts[2].strip(")\""))

        # Create a dictionary for x if it doesn't exist
        if x not in dictionaries:
            dictionaries[x] = {}

        # Add the corresponding d to the dictionary with t as the key
        dictionaries[x][t] = a

    return dictionaries

In [ ]:
# run the orders
def run_orders(env, orders):
    t = 0

    while True:
        dictionary = {}
        for i in orders:
            if t in orders[i]: value = orders[i][t]
            else: value = 0
            dictionary[i] = int(value)
        print(str(t) + ": "+ str(dict(sorted(dictionary.items()))))
    
        obs, rew, done, info = env.step(dictionary)
        render_env(env,True)

        if done["__all__"]:
            return all(info["state"][i] == TrainState.DONE for i in info["state"])

        t += 1

In [ ]:
def run_manually(env):
    c = 0
    while True:
        dictionary = {}
        for i in env.get_agent_handles():
            print("Order for agent " + str(i) + ":")
            order = input()
            if order: dictionary[i] = int(order)
            else: dictionary[i] = 0
    
        input()
        obs, rew, done, info = env.step(dictionary)
        render_env(env,True)


        if done["__all__"]: break

# Execution

In [ ]:
filename = "024x024_004/ex00"

with open("testing/flatland/objects/" + filename + ".pkl", "rb") as file:
    env = pickle.load(file)

with open("testing/flatland/facts/" + filename + ".lp", "r") as file:
    input = file.read()

obs = env.reset()
# deactivate the horizon
env._max_episode_steps = None
render_env(env,True)

# orders = get_orders("testing/flatland/facts/" + filename + ".lp","encodings/multiple/ordered/combined.lp")
sat, time, solve, atoms = run_clingo(input, "encodings/multiple/test/", 600, 10)
# print([a for a in atoms if "state" in a])
orders = facts_to_flatland(atoms)

output = dict(sorted(orders.items()))
for i in output:
    output[i] = dict(sorted(output[i].items()))
print(output)

In [ ]:
# run_manually(env)
print(run_orders(env,orders))